In [1]:
import collections
import gym
import numpy as np
import connect4_gym
import torch as th
from sb3_contrib import MaskablePPO
from sb3_contrib.common.maskable.evaluation import evaluate_policy
from sb3_contrib.common.envs import InvalidActionEnvDiscrete
from sb3_contrib.ppo_mask import MlpPolicy
from stable_baselines3.common.callbacks import BaseCallback
from sb3_contrib.common.maskable.evaluation import evaluate_policy

/home/ml/.cache/pypoetry/virtualenvs/connect4-rl-2Gm6cHT4-py3.9/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [2]:
policy_kwargs = dict(
    activation_fn=th.nn.ReLU,
    net_arch=[dict(pi=[128, 128], vf=[128, 128])]
)

In [3]:
opponents = collections.deque()

In [4]:
def save_opponent(opponent):
    opponent_copy = MlpPolicy(
        model.observation_space,
        model.action_space,
        model.lr_schedule,
        **model.policy_kwargs,
    )
    opponent_copy.load_state_dict(opponent.state_dict())
    opponents.append(opponent_copy)

In [5]:
class SaveOpponentCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.n_iterations = 0
    
    def _on_rollout_start(self):
        save_opponent(self.model.policy)
        self.n_iterations += 1
        if len(opponents) > 1 and len(opponents) > self.n_iterations * 0.5:
            opponents.popleft()
            
    def _on_step(self):
        return True

callback = SaveOpponentCallback()

In [6]:
opponent_rng = np.random.default_rng()

In [7]:
def get_opponent():
    opponent = opponent_rng.choice(opponents)
    return lambda obs, action_masks: opponent.predict(obs, action_masks=action_masks)[0]

In [8]:
move_rng = np.random.default_rng()
def random_opponent(_observation, action_mask):
    valid_moves = action_mask.nonzero()[0]
    return move_rng.choice(valid_moves)

In [9]:
env = gym.make("Connect4-v0", get_opponent=get_opponent)

/home/ml/.cache/pypoetry/virtualenvs/connect4-rl-2Gm6cHT4-py3.9/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:20: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/home/ml/.cache/pypoetry/virtualenvs/connect4-rl-2Gm6cHT4-py3.9/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:25: UserWarning: WARN: It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


In [10]:
model = MaskablePPO(
    "MlpPolicy",
    env,
    policy_kwargs=policy_kwargs,
    learning_rate=0.001,
    n_steps=2048,
    batch_size=512,
    n_epochs=10,
    gamma=0.97,
    tensorboard_log="logs",
    verbose=1,
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [11]:
save_opponent(model.policy)

In [12]:
model.learn(
    100000,
    callback=callback,
    eval_env=gym.make("Connect4-v0", get_opponent=lambda: random_opponent),
    eval_freq=2048,
    n_eval_episodes=100,
    eval_log_path="logs",
)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


2022-08-03 22:18:16.999377: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-03 22:18:16.999398: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/ml/.cache/pypoetry/virtualenvs/connect4-rl-2Gm6cHT4-py3.9/lib/python3.9/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/ml/.cache/pypoetry/virtualenvs/connect4-rl-2Gm6cHT4-py3.9/lib/python3.9/site-packages/keras/utils/image_utils.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/home/ml/.cache/pypoetry/virtualenvs/connect4-rl-2Gm6cHT4-py3.9/l

Logging to logs/PPO_9
Eval num_timesteps=2048, episode_reward=0.48 +/- 0.88
Episode length: 7.19 +/- 2.53
---------------------------------
| eval/              |          |
|    mean_ep_length  | 7.19     |
|    mean_reward     | 0.48     |
| time/              |          |
|    total timesteps | 2048     |
---------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 10.5     |
|    ep_rew_mean     | -0.08    |
| time/              |          |
|    fps             | 363      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
Eval num_timesteps=4096, episode_reward=0.64 +/- 0.77
Episode length: 6.40 +/- 2.79
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 6.4         |
|    mean_reward          | 0.64        |
| time/                   |             |


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 8.78     |
|    ep_rew_mean     | 0.18     |
| time/              |          |
|    fps             | 419      |
|    iterations      | 8        |
|    time_elapsed    | 39       |
|    total_timesteps | 16384    |
---------------------------------
Eval num_timesteps=18432, episode_reward=0.76 +/- 0.65
Episode length: 7.29 +/- 2.09
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 7.29        |
|    mean_reward          | 0.76        |
| time/                   |             |
|    total timesteps      | 18432       |
| train/                  |             |
|    approx_kl            | 0.016408442 |
|    clip_fraction        | 0.161       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.73       |
|    explained_variance   | -0.0268     |
|    learning_rate        | 0.001       |
|    loss                 | 0

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 7.32     |
|    ep_rew_mean     | 0.18     |
| time/              |          |
|    fps             | 421      |
|    iterations      | 15       |
|    time_elapsed    | 72       |
|    total_timesteps | 30720    |
---------------------------------
Eval num_timesteps=32768, episode_reward=0.96 +/- 0.28
Episode length: 5.17 +/- 2.09
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 5.17        |
|    mean_reward          | 0.96        |
| time/                   |             |
|    total timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.022104276 |
|    clip_fraction        | 0.197       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.43       |
|    explained_variance   | -0.0214     |
|    learning_rate        | 0.001       |
|    loss                 | 0

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 5.83     |
|    ep_rew_mean     | 0.38     |
| time/              |          |
|    fps             | 421      |
|    iterations      | 22       |
|    time_elapsed    | 106      |
|    total_timesteps | 45056    |
---------------------------------
Eval num_timesteps=47104, episode_reward=0.68 +/- 0.73
Episode length: 6.39 +/- 2.39
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 6.39        |
|    mean_reward          | 0.68        |
| time/                   |             |
|    total timesteps      | 47104       |
| train/                  |             |
|    approx_kl            | 0.020805174 |
|    clip_fraction        | 0.167       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.984      |
|    explained_variance   | 0.115       |
|    learning_rate        | 0.001       |
|    loss                 | 0

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 5.92     |
|    ep_rew_mean     | 0.46     |
| time/              |          |
|    fps             | 427      |
|    iterations      | 29       |
|    time_elapsed    | 138      |
|    total_timesteps | 59392    |
---------------------------------
Eval num_timesteps=61440, episode_reward=0.78 +/- 0.63
Episode length: 5.98 +/- 2.49
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 5.98        |
|    mean_reward          | 0.78        |
| time/                   |             |
|    total timesteps      | 61440       |
| train/                  |             |
|    approx_kl            | 0.022320133 |
|    clip_fraction        | 0.16        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.707      |
|    explained_variance   | 0.239       |
|    learning_rate        | 0.001       |
|    loss                 | 0

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6.12     |
|    ep_rew_mean     | 0.5      |
| time/              |          |
|    fps             | 429      |
|    iterations      | 36       |
|    time_elapsed    | 171      |
|    total_timesteps | 73728    |
---------------------------------
Eval num_timesteps=75776, episode_reward=0.88 +/- 0.47
Episode length: 5.60 +/- 2.29
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 5.6         |
|    mean_reward          | 0.88        |
| time/                   |             |
|    total timesteps      | 75776       |
| train/                  |             |
|    approx_kl            | 0.025837667 |
|    clip_fraction        | 0.143       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.532      |
|    explained_variance   | 0.235       |
|    learning_rate        | 0.001       |
|    loss                 | 0

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6.53     |
|    ep_rew_mean     | 0.48     |
| time/              |          |
|    fps             | 428      |
|    iterations      | 43       |
|    time_elapsed    | 205      |
|    total_timesteps | 88064    |
---------------------------------
Eval num_timesteps=90112, episode_reward=0.88 +/- 0.47
Episode length: 5.79 +/- 2.26
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 5.79        |
|    mean_reward          | 0.88        |
| time/                   |             |
|    total timesteps      | 90112       |
| train/                  |             |
|    approx_kl            | 0.022874486 |
|    clip_fraction        | 0.134       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.439      |
|    explained_variance   | 0.295       |
|    learning_rate        | 0.001       |
|    loss                 | 0

In [20]:
model.learn(1000000, callback=callback, reset_num_timesteps=False)

Logging to logs/PPO_8
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 10.1     |
|    ep_rew_mean     | 0.88     |
| time/              |          |
|    fps             | 147610   |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 503808   |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 10        |
|    ep_rew_mean          | 0.74      |
| time/                   |           |
|    fps                  | 72367     |
|    iterations           | 2         |
|    time_elapsed         | 6         |
|    total_timesteps      | 505856    |
| train/                  |           |
|    approx_kl            | 0.0178569 |
|    clip_fraction        | 0.0364    |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.0788   |
|    explained_variance   | 0.154     |
|    learning_rate        | 0.001     

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 10.3        |
|    ep_rew_mean          | 0.92        |
| time/                   |             |
|    fps                  | 13137       |
|    iterations           | 11          |
|    time_elapsed         | 39          |
|    total_timesteps      | 524288      |
| train/                  |             |
|    approx_kl            | 0.055623792 |
|    clip_fraction        | 0.0684      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.0396     |
|    explained_variance   | 0.439       |
|    learning_rate        | 0.001       |
|    loss                 | 0.0256      |
|    n_updates            | 2550        |
|    policy_gradient_loss | -0.00453    |
|    value_loss           | 0.0684      |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.66  

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 9.82       |
|    ep_rew_mean          | 0.84       |
| time/                   |            |
|    fps                  | 6991       |
|    iterations           | 21         |
|    time_elapsed         | 77         |
|    total_timesteps      | 544768     |
| train/                  |            |
|    approx_kl            | 0.12863134 |
|    clip_fraction        | 0.0354     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.0302    |
|    explained_variance   | 0.553      |
|    learning_rate        | 0.001      |
|    loss                 | 0.0387     |
|    n_updates            | 2650       |
|    policy_gradient_loss | -0.00836   |
|    value_loss           | 0.118      |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 10.1        |
|    ep_rew_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 10          |
|    ep_rew_mean          | 0.76        |
| time/                   |             |
|    fps                  | 4879        |
|    iterations           | 31          |
|    time_elapsed         | 115         |
|    total_timesteps      | 565248      |
| train/                  |             |
|    approx_kl            | 0.094109386 |
|    clip_fraction        | 0.0339      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.0525     |
|    explained_variance   | 0.427       |
|    learning_rate        | 0.001       |
|    loss                 | 0.0607      |
|    n_updates            | 2750        |
|    policy_gradient_loss | -0.0024     |
|    value_loss           | 0.148       |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 9.46    

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 9.47       |
|    ep_rew_mean          | 0.92       |
| time/                   |            |
|    fps                  | 3795       |
|    iterations           | 41         |
|    time_elapsed         | 154        |
|    total_timesteps      | 585728     |
| train/                  |            |
|    approx_kl            | 0.06351912 |
|    clip_fraction        | 0.0741     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.0954    |
|    explained_variance   | 0.299      |
|    learning_rate        | 0.001      |
|    loss                 | 0.0538     |
|    n_updates            | 2850       |
|    policy_gradient_loss | -0.00562   |
|    value_loss           | 0.139      |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 9.19       |
|    ep_rew_mean

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 10.6        |
|    ep_rew_mean          | 0.76        |
| time/                   |             |
|    fps                  | 801         |
|    iterations           | 441         |
|    time_elapsed         | 1753        |
|    total_timesteps      | 1404928     |
| train/                  |             |
|    approx_kl            | 0.084602416 |
|    clip_fraction        | 0.0785      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.124      |
|    explained_variance   | 0.278       |
|    learning_rate        | 0.001       |
|    loss                 | 0.0497      |
|    n_updates            | 6850        |
|    policy_gradient_loss | -0.00776    |
|    value_loss           | 0.125       |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 10.1    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 10.3        |
|    ep_rew_mean          | 0.88        |
| time/                   |             |
|    fps                  | 792         |
|    iterations           | 451         |
|    time_elapsed         | 1798        |
|    total_timesteps      | 1425408     |
| train/                  |             |
|    approx_kl            | 0.028530285 |
|    clip_fraction        | 0.0353      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.053      |
|    explained_variance   | 0.294       |
|    learning_rate        | 0.001       |
|    loss                 | 0.00906     |
|    n_updates            | 6950        |
|    policy_gradient_loss | -0.00702    |
|    value_loss           | 0.0545      |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.95  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 10.1        |
|    ep_rew_mean          | 0.8         |
| time/                   |             |
|    fps                  | 785         |
|    iterations           | 461         |
|    time_elapsed         | 1841        |
|    total_timesteps      | 1445888     |
| train/                  |             |
|    approx_kl            | 0.025052158 |
|    clip_fraction        | 0.0439      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.0568     |
|    explained_variance   | 0.3         |
|    learning_rate        | 0.001       |
|    loss                 | 0.0278      |
|    n_updates            | 7050        |
|    policy_gradient_loss | -0.00852    |
|    value_loss           | 0.122       |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 10.1    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.7         |
|    ep_rew_mean          | 0.88        |
| time/                   |             |
|    fps                  | 778         |
|    iterations           | 471         |
|    time_elapsed         | 1884        |
|    total_timesteps      | 1466368     |
| train/                  |             |
|    approx_kl            | 0.017686948 |
|    clip_fraction        | 0.0284      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.0503     |
|    explained_variance   | 0.125       |
|    learning_rate        | 0.001       |
|    loss                 | 0.0106      |
|    n_updates            | 7150        |
|    policy_gradient_loss | -0.00224    |
|    value_loss           | 0.0518      |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 9.18    

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 9.48       |
|    ep_rew_mean          | 0.92       |
| time/                   |            |
|    fps                  | 772        |
|    iterations           | 481        |
|    time_elapsed         | 1925       |
|    total_timesteps      | 1486848    |
| train/                  |            |
|    approx_kl            | 0.05782391 |
|    clip_fraction        | 0.0393     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.0521    |
|    explained_variance   | 0.296      |
|    learning_rate        | 0.001      |
|    loss                 | 0.0373     |
|    n_updates            | 7250       |
|    policy_gradient_loss | -0.00775   |
|    value_loss           | 0.106      |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 9.72       |
|    ep_rew_mean

In [22]:
human_env = gym.make("Connect4-v0", render_mode="human")
evaluate_policy(model, human_env, n_eval_episodes=1, deterministic=False)

Enter your move: 3
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪🔴⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪🟡🔴⚪⚪⚪
Enter your move: 3
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪🔴⚪⚪⚪
⚪⚪🟡🔴⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪🟡🔴⚪⚪⚪
⚪⚪🟡🔴⚪⚪⚪
Enter your move: 3
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪🔴⚪⚪⚪
⚪⚪🟡🔴⚪⚪⚪
⚪⚪🟡🔴⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪🔴⚪⚪⚪
⚪⚪🟡🔴⚪⚪⚪
🟡⚪🟡🔴⚪⚪⚪
Enter your move: 3
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪🔴⚪⚪⚪
⚪⚪⚪🔴⚪⚪⚪
⚪⚪🟡🔴⚪⚪⚪
🟡⚪🟡🔴⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪
⚪⚪⚪⚪⚪⚪⚪


(-1.0, 0.0)